## Image Generation Agent with cost Approval

### Scenario:

Build an agent that generates images using the MCP server, but requires approval for "bulk" image generation:

1. Single image request(1 image):Auto approve, generate immediately.
2. Bulk request (>1image): Pause and ask for approval before generating multiple images
3. Explore different publicly available image generation MCP servers.


In [1]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("ADK components imported successfully.")

ADK components imported successfully.


In [2]:
import os

try:
  GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
  print("Authentication & Setup complete.")
except Exception as e:
  print("Error :",e)


Authentication & Setup complete.


In [4]:
retry_config = types.HttpRetryOptions(
  attempts=5,
  exp_base=7,
  initial_delay=1,
  http_status_codes=[429,500,503,504] #Retry on these HTTP errors
)

In [25]:
#Step 1:
#MCP integration with everything server

mcp_image_server = McpToolset(
  connection_params=StdioConnectionParams(
    server_params=StdioServerParameters(
      command="npx" ,#run mcp server via npx
      args=[
        "-y", #argument for npx to auto-confirm install
        "@modelcontextprotocol/server-everything"
      ],
      tool_filter=["getTinyImage"]
    ),
    timeout=30,
  )
)

> ```User asks → Agent calls tool → Tool PAUSES and asks human → Human approves → Tool completes → Agent responds```

In [24]:
#Step 2: Pausable Tool for Bulk approval
IMAGE_THRESHOLD =1

def bulk_approval_gen(
    num_images: int, prompt: str,tool_context :ToolContext
)->dict:
  """ Generating image. Requires approval if ordering more than 1 image(IMAGE_THRESHOLD).
    Args:
        num_images: Number of images to generate.
        prompt: Description of image

    Returns:
        a dictionary which contains the image encoded-base64.
  """

## SCENARIO 1: 1 image - auto approve

  if num_images == IMAGE_THRESHOLD:
    return {
      "status":"approved",
      "oder_id":f"IMG-{num_images}-AUTO",
      "num_images":{num_images},
      "message":f"Image generation, {num_images} images for '{prompt}' auto-approved."
    }


  ##SCENARIO 2: More than 1 -Need Human approval -Pause here

  if not tool_context.tool_confirmation:
    tool_context.request_confirmation(
      hint=f"Too many Images to generate: {num_images} .Do you want to Approve?",
      payload={"num_images":num_images,
               "prompt":prompt
               }
    )
    return {
      "status":"pending",
      "message":f"BULK : Image Generation for {num_images} images requires approval."
    }

 ## SCENARIO 3: The tool is called AGAIN and is now resuming.
 #  Handle approval response -RESUME here.

  if tool_context.tool_confirmation.confirmed:
    return{
      "status":"approved",
      "oder_id":f"IMG-{num_images}-HUMAN",
      "num_images":{num_images},
      "message":f"Image generation, {num_images} images for '{prompt}' auto-approved."
    }
  else:
    return{
      "status":"rejected",
      "message":f"Image Generation Rejected, {num_images} images required."
    }

print("Long Running functions created!!!")


Long Running functions created!!!


In [23]:
#Step 3:
#Creating image agent with MCP integration with 2 TOOLS
image_agent = LlmAgent(
  model=Gemini(model="gemini-2.5-flash-lite",retry_options=retry_config),
  name="image_agent",
  instruction="""You are an image generation assistant.

  When users request to generate images:
  1. use the bulk_approval_gen tool with the number of images and the prompt.
  2. If the status is 'pending', inform the user that approval is required
  3. After receiving the final result, provide a clear summary including:
    -STATUS
    -ID
    -IMAGES
  4. Keep responses concise but informative.


  """,
  tools=[bulk_approval_gen,mcp_image_server]
)


print("Image generation Agent created!")

Image generation Agent created!


In [26]:
# step 4 Resumable App + Runner

session_service = InMemorySessionService()

image_app = App(
  name="image_generator",
  root_agent=image_agent,
  resumability_config=ResumabilityConfig(is_resumable=True),
)
image_runner= Runner(
  app=image_app,
  session_service=session_service,
)
print("Resumable App and Runner complete.")

/var/folders/kw/6d_1c0rj22d1pqvkf0fldhlm0000gn/T/ipykernel_55663/1724921035.py:8: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True),


Resumable App and Runner complete.


In [27]:
#Step 5 Helper Functions & Workflows

In [29]:
#Scans the events for pause (adk_request_confirmation)
def check_for_approval(events):
  for event in events: # scan all the events
    if event.content and event.content.parts: # None check at content
      for part in event.content.parts: # for every event scan every part
        if(
          part.function_call # does it have function call?
          and part.function_call.name == "adk_request_confirmation"): #exact match

          return { #Found return state
            "approval_id":part.function_call.id, #id of the request (for response)
            "invocation_id":event.invocation_id # same id for resume
          }
  return None # not found = no pause

In [30]:
#Prints only the text from the agent

def print_agent_response(events):
  for event in events:
    if event.content and event.content.parts:
      for part in event.content.parts:
        if part.text: #text only
          print(f"AGENT > {part.text}")


In [31]:
# Finds base64 images from MCP tool response and it diplays them
# MCP tool(getTinyImage) --> Returns {'content': [{'type': 'image', 'data': 'base64'}]}

from IPython.display import display, Image as IPImage
import base64

def display_images_from_events(events):
  for event in events:
    if event.content and event.content.parts:
      for part in event.content.parts:
        if hasattr(part,"function_response") and part.function_response:
          for item in part.function_response.response.get("content",[]): #parse json-like
            if item.get("type") == "image": #image type
              img_data = base64.b64decode(item["data"]) #decode base64 -> bytes
              display(IPImage(data=img_data))

In [32]:
#formatting human decision into ADK format(function response)
# ADK requires an exact structure
# without this ADK cant understand the "human approved"

def create_approval_response(approval_info, approved):
  confirmation_response = types.FunctionResponse(
    id = approval_info['approval_id'], #match to request id
    name = 'adk_request_confirmation', #exact name
    response= {"confirmed": approved} #human decision
  )
  return types.Content(
    role="user", parts=[types.Part(function_response=confirmation_response)]
  )

In [ ]:
#main function that orchestrates the workflow, handles Long running operation pause/resume.


